# Transpose Upper Triangular Matrix Vector Multiply Routines

This notebook walks you through how to implement $ x := U^T x $ where $ U $ is upper triangular.  Vector $ y $ is not to be touched (and, indeed, not even passed into the routines).  This is a little trickier than you might think.  You may want to do a few small problems by hand if you don't get the right answer.  Also, PictureFLAME may help you see what is going on.

## Getting started

We will use some functions that are part of our laff library (of which this function will become a part) as well as some routines from the FLAME API (Application Programming Interface) that allows us to write code that closely resembles how we typeset algorithms using the FLAME notation.  These functions are imported with `include` and `using` statements.

## Algorithm that takes dot products

## The `Trmv_ut_unb_var1!( U, x )` routine

This routine, given upper triangular $ U \in \mathbb{R}^{n \times n} $ and $ x \in \mathbb{R}^n $ computes $ x := U^T x $.  The "_ut_" in the name of the routine indicates this is the "upper, transpose" matrix-vector multiplication.

The specific laff functions we will use are 
<ul>
<li> <code> laff.dots!( x, y, alpha ) </code> which computes $ \alpha := x^T y + \alpha $.  </li>
<li> <code> laff.scal!( alpha, x ) </code> which computes $ x := \alpha x $.  </li> (You will want to use this one to update $ \chi_1 $ if you want to use PictureFrame.)
</ul>

Use the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html"> Spark webpage</a> to generate a code skeleton.  (Make sure you adjust the name of the routine.)

<h1> <font color=red> NOTE: The algorithms for this operation march from bottom-right to top-left!!! </font> </h1>

In [17]:
include("../flame.jl")
using .flame
include("../laff/laff.jl")
using .laff

function Trmv_ut_unb_var1!(U, x)

    UTL, UTR, 
    UBL, UBR  = flame.part_2x2(U, 
                               0, 0, "BR")

    xT, 
    xB  = flame.part_2x1(x, 
                         0, "BOTTOM")

    while size(UBR, 1) < size(U, 1)

        U00,  u01,       U02,  
        u10t, upsilon11, u12t, 
        U20,  u21,       U22   = flame.repart_2x2_to_3x3(UTL, UTR, 
                                                         UBL, UBR, 
                                                         1, 1, "TL")

        x0,   
        chi1, 
        x2    = flame.repart_2x1_to_3x1(xT, 
                                        xB, 
                                        1, "TOP")

        #------------------------------------------------------------#

        laff.scal!( upsilon11, chi1 )
        laff.dots!( u01, x0, chi1 )
        
        #------------------------------------------------------------#

        UTL, UTR, 
        UBL, UBR  = flame.cont_with_3x3_to_2x2(U00,  u01,       U02,  
                                               u10t, upsilon11, u12t, 
                                               U20,  u21,       U22,  
                                               "BR")

        xT, 
        xB  = flame.cont_with_3x1_to_2x1(x0,   
                                         chi1, 
                                         x2,   
                                         "BOTTOM")

    end

    flame.merge_2x1!(xT, 
                     xB, x)

end

Trmv_ut_unb_var1! (generic function with 1 method)

## Testing

Let's quickly test the routine by creating a 4 x 4 matrix and related vectors, performing the computation.

In [18]:
U = rand(4, 4)
x = rand(4)
xold = rand(4)

# Notice that U is not upper triangular.  We will only use the upper triangular part.

println( "U before =" )
U

U before =


4×4 Array{Float64,2}:
 0.629388  0.29653    0.903018  0.185783 
 0.720498  0.0753778  0.522167  0.0853234
 0.183003  0.773309   0.287215  0.155395 
 0.734183  0.895674   0.73315   0.103869 

In [19]:
println( "x before =" )
x

x before =


4-element Array{Float64,1}:
 0.4603879306263572 
 0.06921338132715538
 0.2480588575845042 
 0.7895002465587995 

In [20]:
laff.copy!( x, xold )   # save the original vector x

Trmv_ut_unb_var1!( U, x )

println( "x after =" )
x

x after =


4-element Array{Float64,1}:
 0.28976247099263946
 0.14173613583989217
 0.5231258868179769 
 0.2119894598963459 

In [21]:
# UpperTriangular() makes the matrix upper triangular
using LinearAlgebra
println( "x - ( transpose( UpperTriangular( U ) ) * xold ) = " )
x - ( transpose( UpperTriangular( U ) ) * xold )

x - ( transpose( UpperTriangular( U ) ) * xold ) = 


4-element Array{Float64,1}:
 0.0                   
 0.0                   
 0.0                   
 2.7755575615628914e-17

Bingo, it seems to work!  (Notice that we are doing floating point computations, which means that due to rounding you may not get an exact "0".)

## Watch your code in action!

Copy and paste the code into <a href="http://edx-org-utaustinx.s3.amazonaws.com/UT501x/PictureFlame/PictureFLAME.html"> PictureFLAME </a>, a webpage where you can watch your routine in action.  Just cut and paste into the box.  

Disclaimer: we implemented a VERY simple interpreter.  If you do something wrong, we cannot guarantee the results.  But if you do it right, you are in for a treat.

If you want to reset the problem, just click in the box into which you pasted the code and hit "next" again.

## Algorithm that uses axpys

## The `Trmv_ut_unb_var2!( U, x )` routine

This routine, given upper triangular $ U \in \mathbb{R}^{n \times n} $ and $ x \in \mathbb{R}^n $ computes $ x := U^T x $.  The "_ut_" in the name of the routine indicates this is the "upper triangular, transpose" matrix-vector multiplication.  

The specific laff functions we will use are 
<ul>
<li> <code> laff.axpy!( alpha, x, y ) </code> which computes $ y := \alpha x +  y  $.  </li>
<li> <code> laff.scal!( alpha, x ) </code> which computes $ x := \alpha x $.  </li> (You will want to use this one to update $ \chi_1 $ if you want to use PictureFrame.)
</ul>

Use the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html"> Spark webpage</a> to generate a code skeleton.  (Make sure you adjust the name of the routine.)

In [22]:
include("../flame.jl")
using .flame
include("../laff/laff.jl")
using .laff

function Trmv_ut_unb_var2!(U, x)

    UTL, UTR, 
    UBL, UBR  = flame.part_2x2(U, 
                               0, 0, "BR")

    xT, 
    xB  = flame.part_2x1(x, 
                         0, "BOTTOM")

    while size(UBR, 1) < size(U, 1)

        U00,  u01,       U02,  
        u10t, upsilon11, u12t, 
        U20,  u21,       U22   = flame.repart_2x2_to_3x3(UTL, UTR, 
                                                         UBL, UBR, 
                                                         1, 1, "TL")

        x0,   
        chi1, 
        x2    = flame.repart_2x1_to_3x1(xT, 
                                        xB, 
                                        1, "TOP")

        #------------------------------------------------------------#

        laff.axpy!( chi1, u12t, x2 )
        laff.scal!( upsilon11, chi1 )
        
        #------------------------------------------------------------#

        UTL, UTR, 
        UBL, UBR  = flame.cont_with_3x3_to_2x2(U00,  u01,       U02,  
                                               u10t, upsilon11, u12t, 
                                               U20,  u21,       U22,  
                                               "BR")

        xT, 
        xB  = flame.cont_with_3x1_to_2x1(x0,   
                                         chi1, 
                                         x2,   
                                         "BOTTOM")

    end

    flame.merge_2x1!(xT, 
                     xB, x)

end

Trmv_ut_unb_var2! (generic function with 1 method)

## Testing

Let's quickly test the routine by creating a 4 x 4 matrix and related vectors, performing the computation.

In [23]:
U = rand(4, 4)
x = rand(4)
xold = rand(4)

# Notice that U is not upper triangular.  We will only use the upper triangular part.

println( "U before =" )
U

U before =


4×4 Array{Float64,2}:
 0.23073    0.668263   0.425677  0.54395  
 0.417369   0.0776107  0.230134  0.301025 
 0.0488903  0.950892   0.61765   0.0245148
 0.0213001  0.805864   0.994607  0.206901 

In [24]:
println( "x before =" )
x

x before =


4-element Array{Float64,1}:
 0.5863195961520349
 0.9821750964692366
 0.8473921758296192
 0.7449675467003523

In [25]:
laff.copy!( x, xold )   # save the original vector x

Trmv_ut_unb_var2!( U, x )

println( "x after =" )
x

x after =


4-element Array{Float64,1}:
 0.13528179175428237
 0.46804318325233724
 0.9990060092036992 
 0.7894954232984757 

In [26]:
# UpperTriangular() makes the matrix upper triangular
using LinearAlgebra
println( "x - ( transpose( UpperTriangular( U ) ) * xold ) = " )
x - ( transpose( UpperTriangular( U ) ) * xold )

x - ( transpose( UpperTriangular( U ) ) * xold ) = 


4-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0

Bingo, it seems to work!  (Notice that we are doing floating point computations, which means that due to rounding you may not get an exact "0".)

## Watch your code in action!

Copy and paste the code into <a href="http://edx-org-utaustinx.s3.amazonaws.com/UT501x/PictureFlame/PictureFLAME.html"> PictureFLAME </a>, a webpage where you can watch your routine in action.  Just cut and paste into the box.  

Disclaimer: we implemented a VERY simple interpreter.  If you do something wrong, we cannot guarantee the results.  But if you do it right, you are in for a treat.

If you want to reset the problem, just click in the box into which you pasted the code and hit "next" again.